**About Book Crossing Dataset**<br>

This dataset has been compiled by Cai-Nicolas Ziegler in 2004, and it comprises of three tables for users, books and ratings. Explicit ratings are expressed on a scale from 1-10 (higher values denoting higher appreciation) and implicit rating is expressed by 0.

Reference: http://www2.informatik.uni-freiburg.de/~cziegler/BX/ 

**Objective**

This project entails building a Book Recommender System for users based on user-based and item-based collaborative filtering approaches.

In [1]:
#Import all the necessary modules
import pandas as pd
import numpy as np
import os
import seaborn as sns
from surprise import Dataset,Reader
from surprise.model_selection import train_test_split
from surprise import SVD
from surprise import accuracy

import matplotlib.pyplot as plt
%matplotlib inline

from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

#### Execute the below cell to load the datasets

In [2]:
#Loading data
books = pd.read_csv("books.csv", sep=";", error_bad_lines=False, encoding="latin-1")
books.columns = ['ISBN', 'bookTitle', 'bookAuthor', 'yearOfPublication', 'publisher', 'imageUrlS', 'imageUrlM', 'imageUrlL']

b'Skipping line 6452: expected 8 fields, saw 9\nSkipping line 43667: expected 8 fields, saw 10\nSkipping line 51751: expected 8 fields, saw 9\n'
b'Skipping line 92038: expected 8 fields, saw 9\nSkipping line 104319: expected 8 fields, saw 9\nSkipping line 121768: expected 8 fields, saw 9\n'
b'Skipping line 144058: expected 8 fields, saw 9\nSkipping line 150789: expected 8 fields, saw 9\nSkipping line 157128: expected 8 fields, saw 9\nSkipping line 180189: expected 8 fields, saw 9\nSkipping line 185738: expected 8 fields, saw 9\n'
b'Skipping line 209388: expected 8 fields, saw 9\nSkipping line 220626: expected 8 fields, saw 9\nSkipping line 227933: expected 8 fields, saw 11\nSkipping line 228957: expected 8 fields, saw 10\nSkipping line 245933: expected 8 fields, saw 9\nSkipping line 251296: expected 8 fields, saw 9\nSkipping line 259941: expected 8 fields, saw 9\nSkipping line 261529: expected 8 fields, saw 9\n'
F:\Anaconda3\lib\site-packages\IPython\core\interactiveshell.py:2785: Dtyp

In [3]:
users = pd.read_csv('users.csv', sep=';', error_bad_lines=False, encoding="latin-1")
users.columns = ['userID', 'Location', 'Age']

In [4]:
ratings = pd.read_csv('ratings.csv', sep=';', error_bad_lines=False, encoding="latin-1")
ratings.columns = ['userID', 'ISBN', 'bookRating']

### Check no.of records and features given in each dataset

In [5]:
print('Books Dataset')
print('Rows / Columns: ',books.shape[0],'/',books.shape[1])
print('Users Dataset')
print('Rows / Columns: ',users.shape[0],'/',users.shape[1])
print('Ratings Dataset')
print('Rows / Columns: ',ratings.shape[0],'/',ratings.shape[1])


Books Dataset
Rows / Columns:  271360 / 8
Users Dataset
Rows / Columns:  278858 / 3
Ratings Dataset
Rows / Columns:  1149780 / 3


## Exploring books dataset

In [6]:
books.head()

,ISBN,bookTitle,bookAuthor,yearOfPublication,publisher,imageUrlS,imageUrlM,imageUrlL
0,0195153448,Classical Mythology,Mark P. O. Morford,2002,Oxford University Press,http://images.amazon.com/images/P/0195153448.0...,http://images.amazon.com/images/P/0195153448.0...,http://images.amazon.com/images/P/0195153448.0...
1,0002005018,Clara Callan,Richard Bruce Wright,2001,HarperFlamingo Canada,http://images.amazon.com/images/P/0002005018.0...,http://images.amazon.com/images/P/0002005018.0...,http://images.amazon.com/images/P/0002005018.0...
2,0060973129,Decision in Normandy,Carlo D'Este,1991,HarperPerennial,http://images.amazon.com/images/P/0060973129.0...,http://images.amazon.com/images/P/0060973129.0...,http://images.amazon.com/images/P/0060973129.0...
3,0374157065,Flu: The Story of the Great Influenza Pandemic...,Gina Bari Kolata,1999,Farrar Straus Giroux,http://images.amazon.com/images/P/0374157065.0...,http://images.amazon.com/images/P/0374157065.0...,http://images.amazon.com/images/P/0374157065.0...
4,0393045218,The Mummies of Urumchi,E. J. W. Barber,1999,W. W. Norton &amp; Company,http://images.amazon.com/images/P/0393045218.0...,http://images.amazon.com/images/P/0393045218.0...,http://images.amazon.com/images/P/0393045218.0...


### Drop last three columns containing image URLs which will not be required for analysis

In [7]:
books.drop(['imageUrlS','imageUrlM','imageUrlL'],inplace=True,axis=1)

In [8]:
books.head()

,ISBN,bookTitle,bookAuthor,yearOfPublication,publisher
0,0195153448,Classical Mythology,Mark P. O. Morford,2002,Oxford University Press
1,0002005018,Clara Callan,Richard Bruce Wright,2001,HarperFlamingo Canada
2,0060973129,Decision in Normandy,Carlo D'Este,1991,HarperPerennial
3,0374157065,Flu: The Story of the Great Influenza Pandemic...,Gina Bari Kolata,1999,Farrar Straus Giroux
4,0393045218,The Mummies of Urumchi,E. J. W. Barber,1999,W. W. Norton &amp; Company


**yearOfPublication**

### Check unique values of yearOfPublication


In [9]:
books['yearOfPublication'].unique()

array([2002, 2001, 1991, 1999, 2000, 1993, 1996, 1988, 2004, 1998, 1994,
       2003, 1997, 1983, 1979, 1995, 1982, 1985, 1992, 1986, 1978, 1980,
       1952, 1987, 1990, 1981, 1989, 1984, 0, 1968, 1961, 1958, 1974, 1976,
       1971, 1977, 1975, 1965, 1941, 1970, 1962, 1973, 1972, 1960, 1966,
       1920, 1956, 1959, 1953, 1951, 1942, 1963, 1964, 1969, 1954, 1950,
       1967, 2005, 1957, 1940, 1937, 1955, 1946, 1936, 1930, 2011, 1925,
       1948, 1943, 1947, 1945, 1923, 2020, 1939, 1926, 1938, 2030, 1911,
       1904, 1949, 1932, 1928, 1929, 1927, 1931, 1914, 2050, 1934, 1910,
       1933, 1902, 1924, 1921, 1900, 2038, 2026, 1944, 1917, 1901, 2010,
       1908, 1906, 1935, 1806, 2021, '2000', '1995', '1999', '2004',
       '2003', '1990', '1994', '1986', '1989', '2002', '1981', '1993',
       '1983', '1982', '1976', '1991', '1977', '1998', '1992', '1996', '0',
       '1997', '2001', '1974', '1968', '1987', '1984', '1988', '1963',
       '1956', '1970', '1985', '1978', '1973', '1980'

As it can be seen from above that there are some incorrect entries in this field. It looks like Publisher names 'DK Publishing Inc' and 'Gallimard' have been incorrectly loaded as yearOfPublication in dataset due to some errors in csv file.


Also some of the entries are strings and same years have been entered as numbers in some places. We will try to fix these things in the coming questions.

### Check the rows having 'DK Publishing Inc' as yearOfPublication

In [10]:
books[books['yearOfPublication'] == 'DK Publishing Inc']

,ISBN,bookTitle,bookAuthor,yearOfPublication,publisher
209538,078946697X,"DK Readers: Creating the X-Men, How It All Beg...",2000,DK Publishing Inc,http://images.amazon.com/images/P/078946697X.0...
221678,0789466953,"DK Readers: Creating the X-Men, How Comic Book...",2000,DK Publishing Inc,http://images.amazon.com/images/P/0789466953.0...


### Drop the rows having `'DK Publishing Inc'` and `'Gallimard'` as `yearOfPublication`

In [11]:
books.shape
indexNames = books[(books['yearOfPublication'] == 'DK Publishing Inc') | (books['yearOfPublication'] == 'Gallimard')].index
books.drop(indexNames,inplace=True)
books.shape

(271360, 5)

(271357, 5)

### Change the datatype of yearOfPublication to 'int'

In [12]:
books.dtypes

ISBN                 object
bookTitle            object
bookAuthor           object
yearOfPublication    object
publisher            object
dtype: object

In [13]:
books.yearOfPublication = books.yearOfPublication.astype(int)

In [14]:
books.dtypes

ISBN                 object
bookTitle            object
bookAuthor           object
yearOfPublication     int32
publisher            object
dtype: object

### Drop NaNs in `'publisher'` column


In [15]:
#Check for NAN values in dataframe
if books.isna().values.any() == True:
    print('NaN values listed')
    books[books.isna().values]
else:
    print('No NaN values present')

NaN values listed


,ISBN,bookTitle,bookAuthor,yearOfPublication,publisher
128890,193169656X,Tyrant Moon,Elaine Corvidae,2002,NaN
129037,1931696993,Finders Keepers,Linnea Sinclair,2001,NaN
187689,9627982032,The Credit Suisse Guide to Managing Your Perso...,NaN,1995,Edinburgh Financial Publishing


In [16]:
indexNames = books[books.isna().values][books['publisher'].isna()].index
books.drop(indexNames,inplace=True)

F:\Anaconda3\lib\site-packages\ipykernel_launcher.py:1: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  """Entry point for launching an IPython kernel.


## Exploring Users dataset

In [17]:
print(users.shape)
users.head()

(278858, 3)


,userID,Location,Age
0,1,"nyc, new york, usa",NaN
1,2,"stockton, california, usa",18.0
2,3,"moscow, yukon territory, russia",NaN
3,4,"porto, v.n.gaia, portugal",17.0
4,5,"farnborough, hants, united kingdom",NaN


### Get all unique values in ascending order for column `Age`

In [18]:
np.sort(users['Age'].unique())

array([   0.,    1.,    2.,    3.,    4.,    5.,    6.,    7.,    8.,
          9.,   10.,   11.,   12.,   13.,   14.,   15.,   16.,   17.,
         18.,   19.,   20.,   21.,   22.,   23.,   24.,   25.,   26.,
         27.,   28.,   29.,   30.,   31.,   32.,   33.,   34.,   35.,
         36.,   37.,   38.,   39.,   40.,   41.,   42.,   43.,   44.,
         45.,   46.,   47.,   48.,   49.,   50.,   51.,   52.,   53.,
         54.,   55.,   56.,   57.,   58.,   59.,   60.,   61.,   62.,
         63.,   64.,   65.,   66.,   67.,   68.,   69.,   70.,   71.,
         72.,   73.,   74.,   75.,   76.,   77.,   78.,   79.,   80.,
         81.,   82.,   83.,   84.,   85.,   86.,   87.,   88.,   89.,
         90.,   91.,   92.,   93.,   94.,   95.,   96.,   97.,   98.,
         99.,  100.,  101.,  102.,  103.,  104.,  105.,  106.,  107.,
        108.,  109.,  110.,  111.,  113.,  114.,  115.,  116.,  118.,
        119.,  123.,  124.,  127.,  128.,  132.,  133.,  136.,  137.,
        138.,  140.,

Age column has some invalid entries like nan, 0 and very high values like 100 and above

### Values below 5 and above 90 do not make much sense for our book rating case...hence replace these by NaNs

In [19]:
users.Age.iloc[users[(users['Age'] < 5) | (users['Age'] > 90)].index] = np.NaN

F:\Anaconda3\lib\site-packages\pandas\core\indexing.py:189: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self._setitem_with_indexer(indexer, value)


In [20]:
np.sort(users['Age'].unique())

array([  5.,   6.,   7.,   8.,   9.,  10.,  11.,  12.,  13.,  14.,  15.,
        16.,  17.,  18.,  19.,  20.,  21.,  22.,  23.,  24.,  25.,  26.,
        27.,  28.,  29.,  30.,  31.,  32.,  33.,  34.,  35.,  36.,  37.,
        38.,  39.,  40.,  41.,  42.,  43.,  44.,  45.,  46.,  47.,  48.,
        49.,  50.,  51.,  52.,  53.,  54.,  55.,  56.,  57.,  58.,  59.,
        60.,  61.,  62.,  63.,  64.,  65.,  66.,  67.,  68.,  69.,  70.,
        71.,  72.,  73.,  74.,  75.,  76.,  77.,  78.,  79.,  80.,  81.,
        82.,  83.,  84.,  85.,  86.,  87.,  88.,  89.,  90.,  nan])

### Replace null values in column `Age` with mean

In [21]:
users.Age.fillna(np.round(users.Age.mean(),decimals=2),inplace=True)
np.sort(users['Age'].unique())

array([  5.  ,   6.  ,   7.  ,   8.  ,   9.  ,  10.  ,  11.  ,  12.  ,
        13.  ,  14.  ,  15.  ,  16.  ,  17.  ,  18.  ,  19.  ,  20.  ,
        21.  ,  22.  ,  23.  ,  24.  ,  25.  ,  26.  ,  27.  ,  28.  ,
        29.  ,  30.  ,  31.  ,  32.  ,  33.  ,  34.  ,  34.72,  35.  ,
        36.  ,  37.  ,  38.  ,  39.  ,  40.  ,  41.  ,  42.  ,  43.  ,
        44.  ,  45.  ,  46.  ,  47.  ,  48.  ,  49.  ,  50.  ,  51.  ,
        52.  ,  53.  ,  54.  ,  55.  ,  56.  ,  57.  ,  58.  ,  59.  ,
        60.  ,  61.  ,  62.  ,  63.  ,  64.  ,  65.  ,  66.  ,  67.  ,
        68.  ,  69.  ,  70.  ,  71.  ,  72.  ,  73.  ,  74.  ,  75.  ,
        76.  ,  77.  ,  78.  ,  79.  ,  80.  ,  81.  ,  82.  ,  83.  ,
        84.  ,  85.  ,  86.  ,  87.  ,  88.  ,  89.  ,  90.  ])

### Change the datatype of `Age` to `int`

In [22]:
users.Age = users.Age.astype(int)
users.Age.dtype

dtype('int32')

In [23]:
print(sorted(users.Age.unique()))

[5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50, 51, 52, 53, 54, 55, 56, 57, 58, 59, 60, 61, 62, 63, 64, 65, 66, 67, 68, 69, 70, 71, 72, 73, 74, 75, 76, 77, 78, 79, 80, 81, 82, 83, 84, 85, 86, 87, 88, 89, 90]


## Exploring the Ratings Dataset

### check the shape

In [24]:
ratings.shape

(1149780, 3)

In [25]:
n_users = users.shape[0]
n_books = books.shape[0]

In [26]:
ratings.head(2)
users.head(2)
books.head(2)

,userID,ISBN,bookRating
0,276725,034545104X,0
1,276726,0155061224,5


,userID,Location,Age
0,1,"nyc, new york, usa",34
1,2,"stockton, california, usa",18


,ISBN,bookTitle,bookAuthor,yearOfPublication,publisher
0,0195153448,Classical Mythology,Mark P. O. Morford,2002,Oxford University Press
1,0002005018,Clara Callan,Richard Bruce Wright,2001,HarperFlamingo Canada


### Ratings dataset should have books only which exist in our books dataset. Drop the remaining rows

In [27]:
print('Before dropping')
ratings.shape
books_isbn = books.ISBN.unique()
index = ratings.query('ISBN not in @books_isbn').index
print('Number of rows dropped: ',len(index))
ratings.drop(index=index,inplace=True)
print('After dropping')
ratings.shape

Before dropping


(1149780, 3)

Number of rows dropped:  118650
After dropping


(1031130, 3)

### Ratings dataset should have ratings from users which exist in users dataset. Drop the remaining rows

In [28]:
print('Before dropping')
ratings.shape
users_userid = users.userID.unique()
index = ratings.query('userID not in @users_userid').index
print('Number of rows dropped: ',len(index))
ratings.drop(index=index,inplace=True)
print('After dropping')
ratings.shape

Before dropping


(1031130, 3)

Number of rows dropped:  0
After dropping


(1031130, 3)

### Consider only ratings from 1-10 and leave 0s in column `bookRating`

In [29]:
ratings.bookRating.unique()

array([ 0,  5,  3,  6,  7,  9,  8, 10,  1,  4,  2], dtype=int64)

In [30]:
index = ratings[ratings['bookRating'] == 0].index
ratings.drop(index=index,inplace=True)
print('After removing ratings 0')
ratings.bookRating.unique()

After removing ratings 0


array([ 5,  3,  6,  7,  9,  8, 10,  1,  4,  2], dtype=int64)

### Find out which rating has been given highest number of times

In [31]:
ratings.groupby(by='bookRating').count()

,userID,ISBN
bookRating,,
1,1481,1481
2,2375,2375
3,5118,5118
4,7617,7617
5,45355,45355
6,31687,31687
7,66401,66401
8,91804,91804
9,60776,60776


    Rating 8 has been given 91804 times the maximum number of times

### **Collaborative Filtering Based Recommendation Systems**

### For more accurate results only consider users who have rated atleast 100 books

In [32]:
#books[books['yearOfPublication'] == 'DK Publishing Inc']
user_trating = ratings.groupby(by='userID').count()
ratings_userID = user_trating[user_trating.ISBN >= 100].iloc[:,2:2]
ratings_userID = ratings_userID.index
final_ratingslist = ratings.query('userID in @ratings_userID')

### Generating ratings matrix from explicit ratings


#### Note: since NaNs cannot be handled by training algorithms, replace these by 0, which indicates absence of ratings

In [33]:
final_ratingslist.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 103269 entries, 1456 to 1147615
Data columns (total 3 columns):
userID        103269 non-null int64
ISBN          103269 non-null object
bookRating    103269 non-null int64
dtypes: int64(2), object(1)
memory usage: 3.2+ MB


In [34]:
# checking for NaN values in final_ratingslist
if final_ratingslist.isna().values.any == True:
    print(final_ratingslist.isna().values)
else:
    print('No NaN values present')

No NaN values present


### Generate the predicted ratings using SVD with no.of singular values to be 50

In [35]:
reader = Reader(rating_scale=(1,5))
data = Dataset.load_from_df(final_ratingslist[['userID','ISBN','bookRating']],reader)

In [36]:
# split data to train and test
trainset,testset = train_test_split(data,test_size=0.2,random_state=100)

In [37]:
svd_model = SVD(n_factors=50,biased=False)
svd_model.fit(trainset)

In [38]:
test_pred = svd_model.test(testset)

In [39]:
test_pred_df = pd.DataFrame([[x.uid,x.iid,x.est] for x in test_pred])
test_pred_df.columns = ['userID','ISBN','estRating']
test_pred_df.sort_values(by= ['userID','estRating'],ascending=False,inplace=True)

In [40]:
# predicted matrix
test_pred_df.head()

,userID,ISBN,estRating
836,278418,0060392185,5.0
3173,278418,0312923651,5.0
3937,278418,0812531558,5.0
3975,278418,0446520608,5.0
7010,278418,0812543181,5.0


### Take a particular user_id

### Lets find the recommendations for user with id `2110`

#### Note: Execute the below cells to get the variables loaded

In [41]:
userID = 2110

In [42]:
user_id = 2 #2nd row in ratings matrix and predicted matrix

### Get the predicted ratings for userID `2110` and sort them in descending order

In [43]:
test_pred_df.sort_values(by = 'estRating',ascending=False,inplace=True)
test_pred_df[test_pred_df.userID == 2110]

,userID,ISBN,estRating
14667,2110,0671802283,5.000000
19737,2110,0440920221,5.000000
19073,2110,0373638078,5.000000
17799,2110,0394563131,5.000000
16972,2110,0373642849,5.000000
16565,2110,0671038931,5.000000
15776,2110,059046678X,5.000000
13812,2110,0886774500,5.000000
13234,2110,1570420564,5.000000
12502,2110,0590880748,5.000000


### Create a dataframe with name `user_data` containing userID `2110` explicitly interacted books

In [44]:
user_data = final_ratingslist[final_ratingslist.userID == 2110]

In [45]:
user_data.head()

,userID,ISBN,bookRating
14448,2110,0060987529,7
14449,2110,0064472779,8
14450,2110,0140022651,10
14452,2110,0142302163,8
14453,2110,0151008116,5


In [46]:
user_data.shape

(103, 3)

### Combine the user_data and and corresponding book data(`book_data`) in a single dataframe with name `user_full_info`

In [47]:
ISBNlist = user_data.ISBN.unique()
ISBNlist = pd.DataFrame(ISBNlist)
ISBNlist.columns = ['ISBN']

ratingtemp = ratings.query('ISBN in @ISBNlist.ISBN')
ratingtemp = ratingtemp.query('userID in [2110]')
#ratingtemp

usertemp = books.query('ISBN in @ISBNlist.ISBN')
usertemp['userID']=2110
usertemp['Location']='charlotte, north carolina, usa'
usertemp['Age']=34
user_full_info =pd.merge(usertemp,ratingtemp,on='ISBN')
user_full_info.drop('userID_y',axis=1,inplace=True)
user_full_info.rename(columns={'userID_x':'user_ID'},inplace=True)

F:\Anaconda3\lib\site-packages\ipykernel_launcher.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  # Remove the CWD from sys.path while we load stuff.
F:\Anaconda3\lib\site-packages\ipykernel_launcher.py:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  # This is added back by InteractiveShellApp.init_path()
F:\Anaconda3\lib\site-packages\ipykernel_launcher.py:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the doc

In [48]:
user_full_info.head()

,ISBN,bookTitle,bookAuthor,yearOfPublication,publisher,user_ID,Location,Age,bookRating
0,0151008116,Life of Pi,Yann Martel,2002,Harcourt,2110,"charlotte, north carolina, usa",34,5
1,015216250X,So You Want to Be a Wizard: The First Book in ...,Diane Duane,2001,Magic Carpet Books,2110,"charlotte, north carolina, usa",34,8
2,0064472779,All-American Girl,Meg Cabot,2003,HarperTrophy,2110,"charlotte, north carolina, usa",34,8
3,0345307674,Return of the Jedi (Star Wars),James Kahn,1983,Del Rey Books,2110,"charlotte, north carolina, usa",34,10
4,0671527215,Hitchhikers's Guide to the Galaxy,Douglas Adams,1984,Pocket,2110,"charlotte, north carolina, usa",34,9


### Get top 10 recommendations for above given userID from the books not already rated by that user

In [49]:
test_pred_df.sort_values(by = 'estRating',ascending=False,inplace=True)

In [50]:
test_pred_df[test_pred_df.userID == 2110]

,userID,ISBN,estRating
7146,2110,0142302163,5.000000
9568,2110,0590448285,5.000000
6636,2110,0590956159,5.000000
4205,2110,042516540X,5.000000
14667,2110,0671802283,5.000000
19737,2110,0440920221,5.000000
19073,2110,0373638078,5.000000
17799,2110,0394563131,5.000000
16972,2110,0373642849,5.000000
16565,2110,0671038931,5.000000
